# JUPYTER NOTEBOOK FOR HOMEWORK 2 - question 1
Extracting texture features and matching them for scene classification

In [61]:
import os, glob, time, sys, cv2
import pandas as pd
from tqdm import tqdm

from skimage import io, transform
from skimage.filters.edges import convolve
import numpy as np
import joblib
from skimage import color
import matplotlib.pyplot as plt

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.preprocessing import MinMaxScaler #use for data normalization

from makeLMfilters import makeLMfilters
np.random.seed(42)

## 1. Load the training and testing images from the two .CSV files respectively

In [47]:
#Load the data along with the class labels
# Note, you will need to use the file folder name to determine the label of an image
# Convert the images to grayscale and resize them to 128 x 128
# (Optional) You might want to save your data here (to avoid repeating this step)
'*** Enter your code below'
df = pd.read_csv("train250.csv").to_numpy()
train = []
imgs = df[:,:1]
ytrain = df[:,1]
for i in imgs:
    img = cv2.imread(i[0])
    img = cv2.resize(img, (128, 128))
    img = color.rgb2gray(img)
    train.append(img)
train = np.array(train)
df = pd.read_csv("test250.csv").to_numpy()
test = []
imgs = df[:,:1]
ytest = df[:,1]
for i in imgs:
    img = cv2.imread(i[0])
    img = cv2.resize(img, (128, 128))
    img = color.rgb2gray(img)
    test.append(img)
test = np.array(test)
print(train.shape)
print(test.shape)
print(ytrain.shape)
print(ytest.shape)
#for i in images[:10]:
#    plt.imshow(img)
#    plt.show()


(250, 128, 128)
(250, 128, 128)
(250,)
(250,)


## 2. Apply a bank of filters and generate features for training, testing and save the dataset

In [48]:
#. Using the overcomplete bank of filters F provided, convolve your input image with each filter in the bank
#  (make sure you check the dimensions of the returned array)
#
def getAbsFilterResponses( F, imggray ):
    rows, cols = imggray.shape
    num_filters = F.shape[2]
    responses = np.zeros((rows, cols, num_filters))

    #. Convolve the input image with every filter in the bank of filters 
    #   to get a response array; take the absolute value of the values in the array
    '*** Enter your code below'
    for i in range(num_filters):
        f = F[:,:,i]
        tmp = convolve(imggray, f)
        a = abs(np.array(tmp))
        tmp = list(a)
        for j in range(len(tmp)):
            for k in range(len(tmp[0])):
                responses[:,:,i][j][k] = tmp[j][k]
    
    return responses


In [49]:
def apply_filter_bank( df, desc ):
    F = makeLMfilters() # This creates the over-complete bank of filters F
    X = np.array([])    # Initialize the numpy array to stack the responses for each image
    y = []              # The label for each image
    X = X.tolist()
    for i in df:
        X.append(getAbsFilterResponses(F, i))
    if desc == "Training -> ":
        y = ytrain
    if desc == "Testing -> ":
        y = ytest
    return np.array(X), y

In [50]:
## generate training and testing features, save them in a pickle with joblib
# NOTE: The code in this cell has been written for you
# NOTE2: This process can take a VERY LONG TIME...
#
X_train, y_train = apply_filter_bank( train, 'Training -> ' )
X_test, y_test = apply_filter_bank( test, 'Testing -> ' )
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
# save the dataset, pickle using joblib
dataset = f'data.pkl'
obj = {
    'train': {
        'features': X_train,
        'label': y_train
    },
    'test': {
        'features': X_test,
        'label': y_test
    }
}

with open( dataset, 'wb' ) as f:
    joblib.dump( obj, f, compress=3 )

(250, 128, 128, 48) (250,)
(250, 128, 128, 48) (250,)


Load back the dataset (optional)
## 3. Run Kmeans with k=500 (recommended)
save the model again (optional)

In [51]:
# load back the dataset (optional)
# NOTE: The code in this cell has been written for you
dataset = f'data.pkl'
with open( dataset, 'rb' ) as f:
    obj = joblib.load( f )
print ( 'finished loading pkl file back')
    
X_train, y_train = obj['train']['features'], obj['train']['label']
X_test, y_test = obj['test']['features'], obj['test']['label']
print( f'X_train shape: {X_train.shape}; y_train shape: {len(y_train)}; X_test shape: {X_test.shape}; y_test shape: {len(y_test)}' )

finished loading pkl file back
X_train shape: (250, 128, 128, 48); y_train shape: 250; X_test shape: (250, 128, 128, 48); y_test shape: 250


In [52]:
# Vectorize the training set 
# NOTE: The vectorization code has been written for you below
reshape_X_train = X_train.reshape( -1, 48 )
print( f'Training dataset shape: {reshape_X_train.shape}' )

# normalize the the reshaped data (recommended to use MinMaxScaler)
'*** Enter your code below' 
scaler = MinMaxScaler()
Xscaled = scaler.fit_transform(reshape_X_train)

# run kmeans (because the data is large, recommended to run minibatch kmeans)
# NOTE: This process can take a VERY LONG TIME...
'*** Enter your code below'
mbk = MiniBatchKMeans(n_clusters=500, batch_size=1000,
                      n_init=10, max_no_improvement=10, verbose=0)
t0 = time.time()
mbk.fit(Xscaled)
t = time.time() - t0
print(t)

# save kmeans model (optional)
# NOTE: The data saving code has been written for you below
model = f'kmeans_model.pkl'
with open( model, 'wb' ) as f:
    joblib.dump( mbk, f, compress=3 )
print( f'Kmeans model saved.' )

Training dataset shape: (4096000, 48)
139.1418755054474
Kmeans model saved.


## 4. Generate the frequency histogram (feature) for each image

In [53]:
# load the model (optional)
model = f'kmeans_model.pkl'
with open( model, 'rb' ) as f:
    mbk = joblib.load( f )

In [54]:
# Given an image response (already filtered) and the kmeans model, compute its frequency histogram
# a) Vectorize the response image
# b) Max-min normalize the response
# c) Predict clusters for each 48-dimensional pixel using the k-means predict function
# d) Generate the frequency histogram by incrementing the cluster centers where each pixel belongs
# e) Normalize the resulting frequency histogram so that it sums to 1
def calc_freq( img, mbk ):
    '*** Enter your code below'
    imgreshape = img.reshape(-1, 48)
    normalImg = scaler.fit_transform(imgreshape)
    prediction = mbk.predict(normalImg)
    hist = np.histogram(prediction, bins = 500)
    img_hist = scaler.fit_transform(hist[0].reshape(-1,1))
    
    return img_hist #this is the histogram feature representing an image

In [73]:
# loop through the set and call calc_freq on each image in the set (training and testing)
'*** Enter your code below' 
trainfreq = []
for i in tqdm(range(len(X_train))):
    trainfreq.append(calc_freq(X_train[i,:,:,:], mbk))
trainfreq = np.array(trainfreq)

testfreq = []
for i in tqdm(range(len(X_test))):
    testfreq.append(calc_freq(X_test[i,:,:,:], mbk))
testfreq = np.array(testfreq)


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:28<00:00,  8.85it/s]

[0.06366723]
(250, 500, 1)


0.0636672325976231
(250, 500, 1)
0.0030977131828419926


## 5. Implement the "nearest neighbor classifier" by comparing each image in Testing with the reference training images to generate a label.
Compare using chi-square distance measure

In [56]:
# Implement a function to calculate chi-square distance between 2 histograms ref and img
#
def calc_chi_sq( ref, img ):
    '*** Enter your code below'
    dist = 0.5 * np.sum(np.divide(np.square(img - ref), img + ref + 1e-10))
    return dist

In [57]:
# Using the chi-sq distances between each img in training
# and each image in test, find the closest match for each test img in training, 
#  assign the label of the closest match to that test img
# Your output will be 'y_pred', the array of predicted test labels.
'*** Enter your code below'
testprediction = []
for i in tqdm(range(len(testfreq))):
    match = -1
    distance = 99999
    for j in tqdm(range(len(trainfreq))):
        dist = calc_chi_sq(testfreq[i], trainfreq[j])
        if dist < distance:
            match = j
            distance = dist
    testprediction.append(y_train[match])
    
testprediction = np.array(testprediction)
print(np.mean(y_test == testprediction))



100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 50178.30it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 45616.04it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 83591.84it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62732.64it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 45242.09it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62698.88it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62400.38it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 83638.51it/s]

100%|███████████████████████████

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 39399.41it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 60928.30it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62683.88it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 50337.29it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 41800.92it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 35809.58it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 35809.58it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 41785.92it/s]

100%|███████████████████████████

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 40759.39it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62642.69it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 50106.37it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 54296.60it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 41795.92it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 50108.76it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62691.38it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 41792.59it/s]

100%|███████████████████████████

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62665.15it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 50180.70it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 83531.90it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62687.63it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 50135.12it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 62661.41it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 41780.93it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 50147.11it/s]

100%|███████████████████████████

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 44124.56it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 41780.93it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 50130.32it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 54621.87it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 250/250 [00:01<00:00, 126.99it/s]


0.28


## 6. Generate a class confusion matrix and print out your accuracy specifically

In [71]:
# Using y_test (true labels on the test data) and y_pred (the predicted labels), 
#  call the appropriate library function to display your confusion matrix, 
#  ensuring that the labels corectly match up with the folder names given
'*** Enter your code below'
print(classification_report(y_test, testprediction))
cm = multilabel_confusion_matrix(y_test, testprediction)
labels = sorted(list(set(y_test)))
for i in range(len(cm)):
    print(labels[i])
    print(cm[i])

              precision    recall  f1-score   support

     01beach       0.43      0.12      0.19        25
    02forest       0.41      0.48      0.44        25
  03mountain       0.26      0.28      0.27        25
      04city       0.29      0.24      0.26        25
    05suburb       0.18      0.48      0.27        25
    06street       0.36      0.52      0.43        25
   07bedroom       0.35      0.24      0.29        25
   08kitchen       0.11      0.04      0.06        25
09livingroom       0.20      0.08      0.11        25
     10store       0.28      0.32      0.30        25

    accuracy                           0.28       250
   macro avg       0.29      0.28      0.26       250
weighted avg       0.29      0.28      0.26       250

01beach
[[221   4]
 [ 22   3]]

02forest
[[208  17]
 [ 13  12]]

03mountain
[[205  20]
 [ 18   7]]

04city
[[210  15]
 [ 19   6]]

05suburb
[[172  53]
 [ 13  12]]

06street
[[202  23]
 [ 12  13]]

07bedroom
[[214  11]
 [ 19   6]]

08kitchen
